In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import os
from scipy import misc
from os import listdir

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import time
import os
import argparse
import torchvision
## Load the model 
#model_conv = torchvision.models.inception_v3(pretrained='imagenet',aux_logits=False)

In [6]:
t=np.array([[1,1,1],[2,2,2],[3,3,3]])
t=Variable(torch.from_numpy(t))

In [44]:
for i, param in model_conv.named_parameters():
    param.requires_grad = False
ct = []
for name, child in model_conv.named_children():
    if "Mixed_7c" in ct:
        for params in child.parameters():
            params.requires_grad = True
    ct.append(name)

model_conv.fc = nn.Linear(2048, 30)

In [45]:
model_conv.AuxLogits.fc=nn.Linear(19200, 30)

In [46]:
for params in model_conv.AuxLogits.fc.parameters():
    params.requires_grad = True

In [42]:
model_conv.AuxLogits.fc

Linear(in_features=19200, out_features=30, bias=True)

In [ ]:
## Lets freeze the first few layers. This is done in two stages 
# Stage-1 Freezing all the layers 
if freeze_layers:
    for i, param in model_conv.named_parameters():
        param.requires_grad = False
# Since imagenet as 1000 classes , We need to change our last layer according to the number of classes we have,
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, n_class)

# Stage-2 , Freeze all the layers till "Conv2d_4a_3*3"
ct = []
for name, child in model_conv.named_children():
    if "Conv2d_4a_3x3" in ct:
        for params in child.parameters():
            params.requires_grad = True
    ct.append(name)
    
# To view which layers are freeze and which layers are not freezed:
for name, child in model_conv.named_childeren():
    for name_2, params in child.named_parameters():
        print(name_2, params.requires_grad)

In [23]:


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5,stride=3)
        self.conv2 = nn.Conv2d(6, 16, 5,stride=3)
        self.fc1   = nn.Linear(2704, 120)
#         16*5*5
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 30)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.softmax(self.fc3(out))
        return out

In [4]:
class sensenet(nn.Module):
    def __init__(self):
        super(sensenet, self).__init__()
        self.glob_conv_1x1=nn.Conv2d(3,6,1)
        
        self.glob_conv_3x3=nn.Conv2d(6,256,3)
        self.avgpool_3=nn.AvgPool2d(254)
        
        self.glob_conv_5x5=nn.Conv2d(6,256,5)
        self.avgpool_5=nn.AvgPool2d(300)
        
        self.glob_conv_7x7=nn.Conv2d(6,256,7)
        self.avgpool_7=nn.AvgPool2d(300)
        
        
        self.detail_conv_3x3_s2_a=BasicConv2d(3, 6, kernel_size=3, stride=2)
        self.detail_conv_3x3_s2_b=BasicConv2d(6, 36, kernel_size=3, stride=2)
        self.detail_conv_3x3_c=BasicConv2d(36, 216, kernel_size=3)
        self.detail_conv_3x3_d=BasicConv2d(216, 512, kernel_size=3)
        
        self.glob_fc=nn.Linear(100,30)
#         self.conv1 = nn.Conv2d(3, 6, 5,stride=3)
#         self.conv2 = nn.Conv2d(6, 16, 5,stride=3)
#         self.fc1   = nn.Linear(2704, 120)
# #         16*5*5
#         self.fc2   = nn.Linear(120, 84)
#         self.fc3   = nn.Linear(84, 30)

    def forward(self, x):
        
        g_out=self.glob_conv_1x1(x)
        
        g_out_3=self.glob_conv_3x3(g_out)
        avg_3 = self.avgpool_3(g_out_3)
        
        g_out_5=self.glob_conv_5x5(g_out)
        avg_5 = self.avgpool_5(g_out_5)
        
        g_out_7=self.glob_conv_7x7(g_out)
        avg_7 = self.avgpool_7(g_out_7)
        
        g_avg=torch.cat((avg_3,avg_5), 0)
        g_avg=torch.cat((g_avg,avg_7), 0)
        
        d_out=self.detail_conv_3x3_s2_a(x)
        d_out=self.detail_conv_3x3_s2_b(dout)
        
        d_out=self.detail_conv_3x3_c(dout)
        d_out = F.max_pool2d(d_out, kernel_size=3)
        
        d_out=self.detail_conv_3x3_d(dout)
        d_out = F.max_pool2d(d_out, kernel_size=3)
        
        d_out = d_out.view(d_out.size(0), -1)
        
        final_vec=torch.cat((g_avg,d_out), 0)
        
        out=self.glob_fc=nn.Linear(final_vec)
        return out
    
class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

In [3]:
batch_size = 120
num_classes = 30
img_width, img_height = 512, 512  # input image dimensions

PATH = os.path.abspath('/home/yicheng/DS/scraper/styles_cropped')
class_lst = listdir(PATH) # all string filenames in a list
path_dic = {}
for Class in class_lst:
    SOURCE_IMAGES = os.path.join(PATH, Class)
    images = [SOURCE_IMAGES+'/'+f for f in listdir(SOURCE_IMAGES) ]
    path_dic[Class] = images

In [4]:
map_class_to_index={}
for i in range(len(class_lst)):
    map_class_to_index[class_lst[i]]=i

In [5]:
def data_loader(batch_size,class_lst,select_index):
    target_values=[]
    img_mats=[]
    counter=1
    for n in select_index:
        for style in class_lst:
            mat=misc.imread(path_dic[style][n])
            mat=np.array(mat)
            if len(mat.shape)<3:
                mat=np.expand_dims(mat, axis=2)
                mat=np.repeat(mat,3,axis=2)
            elif mat.shape[2]==1:
                mat=np.repeat(mat,3,axis=2)
            elif mat.shape[2]>3:
                mat=mat[:,:,:3]
            #mat=mat[:256,:256,:]
            mat=np.swapaxes(mat,0,2)
            mat=np.expand_dims(mat, axis=0)
            img_mats.append(mat)
            target_values.append(map_class_to_index[style])
        
        if counter % batch_size==0:
            #yield (np.asarray(img_mats),np.asarray(target_values))
            yield (np.concatenate(img_mats,axis=0),np.array(target_values))
            target_values=[]
            img_mats=[]
        counter+=1

In [8]:
train_loader=data_loader(1,class_lst,np.arange(1000))


In [23]:
for i in train_loader:
    print(i[0].shape)
    break

(30, 3, 256, 256)


In [9]:
class sensenet(nn.Module):
    def __init__(self):
        super(sensenet, self).__init__()
        self.glob_conv_1x1=nn.Conv2d(3,3,3,stride=3)
        
        self.glob_conv_3x3=nn.Conv2d(3,128,3)

        
        self.glob_conv_5x5=nn.Conv2d(3,128,5)


        self.glob_conv_7x7=nn.Conv2d(3,128,7)

        
        
        self.detail_conv_3x3_s2_a=BasicConv2d(3, 6, kernel_size=4, stride=3)
        self.detail_conv_3x3_s2_b=BasicConv2d(6, 36, kernel_size=3, stride=2)
        self.detail_conv_3x3_c=BasicConv2d(36, 128, kernel_size=3)
        self.detail_conv_3x3_d=BasicConv2d(128, 216, kernel_size=3)
        self.glob_fc_1=nn.Linear(14208,30)
#         self.glob_fc_2=nn.Linear(1000,300)
#         self.glob_fc_3=nn.Linear(300,30)
#         self.conv1 = nn.Conv2d(3, 6, 5,stride=3)
#         self.conv2 = nn.Conv2d(6, 16, 5,stride=3)
#         self.fc1   = nn.Linear(2704, 120)
# #         16*5*5
#         self.fc2   = nn.Linear(120, 84)
#         self.fc3   = nn.Linear(84, 30)

    def forward(self, x):
        
        g_out=self.glob_conv_1x1(x)
        g_out_3=self.glob_conv_3x3(g_out)
        g_avg_3=g_out_3.mean(2,keepdim=True)
        g_avg_3=g_avg_3.mean(3,keepdim=True)
        g_avg_3=g_avg_3.squeeze(2)
        g_avg_3=g_avg_3.squeeze(2)


        g_out_5=self.glob_conv_5x5(g_out)
        g_avg_5=g_out_5.mean(2,keepdim=True)
        g_avg_5=g_avg_5.mean(3,keepdim=True)
        g_avg_5=g_avg_5.squeeze(2)
        g_avg_5=g_avg_5.squeeze(2)

        g_out_7=self.glob_conv_7x7(g_out)
        g_avg_7=g_out_7.mean(2,keepdim=True)
        g_avg_7=g_avg_7.mean(3,keepdim=True)
        g_avg_7=g_avg_7.squeeze(2)
        g_avg_7=g_avg_7.squeeze(2)

        g_avg=torch.cat((g_avg_3,g_avg_5), 1)
        g_avg=torch.cat((g_avg,g_avg_7), 1)
        
#        out=g_avg
        
        d_out=self.detail_conv_3x3_s2_a(x)
        d_out=self.detail_conv_3x3_s2_b(d_out)
        
        d_out=self.detail_conv_3x3_c(d_out)
        d_out = F.max_pool2d(d_out, kernel_size=3)
        
        d_out=self.detail_conv_3x3_d(d_out)
        d_out = F.max_pool2d(d_out, kernel_size=3)
        #print(d_out.size())
        d_out = d_out.view(d_out.size(0), -1)
        #print(d_out.size())
        final_vec=torch.cat((g_avg,d_out), 1)
        
        #print(final_vec.size())
        out=self.glob_fc_1(final_vec)
#         out=F.Sigmoid(out)
#         out=self.glob_fc_2(out)
        out=F.sigmoid(out)
#         out=self.glob_fc_3(out)
        return out
    
class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

In [8]:
class SNet(nn.Module):
    def __init__(self):
        super(SNet, self).__init__()
        self.glob_conv_1x1=nn.Conv2d(3,3,2,stride=2)
        
        self.glob_conv_3x3=nn.Conv2d(3,64,3)

        
        self.glob_conv_5x5=nn.Conv2d(3,64,5)


        #self.glob_conv_7x7=nn.Conv2d(3,64,7)

        
        
        self.detail_conv_3x3_s2_a=BasicConv2d(3, 32, kernel_size=3, stride=2)
        self.detail_conv_3x3_s2_b=BasicConv2d(32, 64, kernel_size=3, stride=2)
        self.inception_1=Inception(64,256)
        self.inception_2=Inception(64,256)
        self.inception_3=Inception(64,256)
        self.inception_4=Inception_B(64,256)
#         self.detail_conv_3x3_c=BasicConv2d(64, 128, kernel_size=3)
#         self.detail_conv_3x3_d=BasicConv2d(128, 216, kernel_size=3)
        self.glob_fc_1=nn.Linear(3264,30)
#         self.glob_fc_2=nn.Linear(1000,300)
#         self.glob_fc_3=nn.Linear(300,30)
#         self.conv1 = nn.Conv2d(3, 6, 5,stride=3)
#         self.conv2 = nn.Conv2d(6, 16, 5,stride=3)
#         self.fc1   = nn.Linear(2704, 120)
# #         16*5*5
#         self.fc2   = nn.Linear(120, 84)
#         self.fc3   = nn.Linear(84, 30)

    def forward(self, x):
        
        g_out=self.glob_conv_1x1(x)
        g_out_3=self.glob_conv_3x3(g_out)
        g_avg_3=g_out_3.mean(2,keepdim=True)
        g_avg_3=g_avg_3.mean(3,keepdim=True)
        g_avg_3=g_avg_3.squeeze(2)
        g_avg_3=g_avg_3.squeeze(2)


        g_out_5=self.glob_conv_5x5(g_out)
        g_avg_5=g_out_5.mean(2,keepdim=True)
        g_avg_5=g_avg_5.mean(3,keepdim=True)
        g_avg_5=g_avg_5.squeeze(2)
        g_avg_5=g_avg_5.squeeze(2)

#         g_out_7=self.glob_conv_7x7(g_out)
#         g_avg_7=g_out_7.mean(2,keepdim=True)
#         g_avg_7=g_avg_7.mean(3,keepdim=True)
#         g_avg_7=g_avg_7.squeeze(2)
#         g_avg_7=g_avg_7.squeeze(2)

        g_avg=torch.cat((g_avg_3,g_avg_5), 1)
#         g_avg=torch.cat((g_avg,g_avg_7), 1)
        
#        out=g_avg
        
        d_out=self.detail_conv_3x3_s2_a(x)
        d_out=self.detail_conv_3x3_s2_b(d_out)
        d_out = F.max_pool2d(d_out, kernel_size=2)
        
        d_out=self.inception_1(d_out)
        d_out=self.inception_2(d_out)
        d_out=self.inception_3(d_out)
        
        d_out = F.max_pool2d(d_out, kernel_size=2)
        
        d_out=self.inception_4(d_out)
        d_out = F.max_pool2d(d_out, kernel_size=2)
        d_out=self.inception_4(d_out)
        
        d_out = F.max_pool2d(d_out, kernel_size=2)
        
#         d_out=self.detail_conv_3x3_c(d_out)
#         d_out = F.max_pool2d(d_out, kernel_size=3)
        
#         d_out=self.detail_conv_3x3_d(d_out)
#         d_out = F.max_pool2d(d_out, kernel_size=3)
#         #print(d_out.size())
        d_out = d_out.view(d_out.size(0), -1)
        #print(d_out.size())
        final_vec=torch.cat((g_avg,d_out), 1)
        
        #print(final_vec.size())
        out=self.glob_fc_1(final_vec)
#         out=F.Sigmoid(out)
#         out=self.glob_fc_2(out)
        out=F.sigmoid(out)
#         out=self.glob_fc_3(out)
        return out
    
class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)
    
class Inception(nn.Module):
    
    def __init__(self,in_channels,out_channels=256,**kwargs):
        super(Inception, self).__init__()
        self.conv1x1_s=BasicConv2d(in_channels,64,kernel_size=1)
        self.conv3x3=BasicConv2d(64,out_channels,kernel_size=3,padding=1)
        self.conv5x5=BasicConv2d(64,out_channels,kernel_size=5,padding=2)
        self.conv1x1_e=BasicConv2d(576,64,kernel_size=1)
    def forward(self,x):
        m=self.conv1x1_s(x)
        b1=self.conv3x3(m)
        b2=self.conv5x5(m)
        b3=F.avg_pool2d(x,kernel_size=3,stride=1,padding=1)
        
        out=[b1,b2,b3]
        out=torch.cat(out,dim=1)
        out=self.conv1x1_e(out)
        return out
    
class Inception_B(nn.Module):
    
    def __init__(self,in_channels,out_channels=256,**kwargs):
        super(Inception_B, self).__init__()
        self.conv1x1_s=BasicConv2d(in_channels,64,kernel_size=1)
        self.conv3x3=BasicConv2d(64,out_channels,kernel_size=3,padding=1)
        self.conv1x5=BasicConv2d(64,out_channels,kernel_size=[1,5],padding=[0,2])
        self.conv5x1=BasicConv2d(out_channels,out_channels,kernel_size=[5,1],padding=[2,0])
        self.conv1x1_e=BasicConv2d(576,64,kernel_size=1)
    def forward(self,x):
        m=self.conv1x1_s(x)
        
        b1=self.conv3x3(m)
        
        b2=self.conv1x5(m)
        b2=self.conv5x1(b2)
        
        b3=F.avg_pool2d(x,kernel_size=3,stride=1,padding=1)
        
        out=[b1,b2,b3]
        out=torch.cat(out,dim=1)
        out=self.conv1x1_e(out)
        return out

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(SNet())

2550149

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(sensenet())

751918

In [9]:
cuda=True
lr=0.001
momentum=0.5
log_interval=10
epochs=70
batch_size=1
# model = LeNet()
#model=model_conv

model=SNet()
if cuda:
    model.cuda()

optimizer = optim.SGD(list(filter(lambda p: p.requires_grad, model.parameters())), lr=lr, momentum=momentum)

def train(epoch):
    model.train()
    for batch_idx, (input_data, input_target) in enumerate(train_loader):
        print(batch_idx)
        data=torch.FloatTensor(input_data)

        target=torch.LongTensor(input_target)

        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        #print(output)
        CEloss = nn.CrossEntropyLoss()
        loss = CEloss(output, target)
        #print(output.data)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), 750*30,
                100. * batch_idx / (750/batch_size), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for input_data, input_target in test_loader:
        
        data=torch.FloatTensor(input_data)

        target=torch.LongTensor(input_target)

        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        loss = nn.CrossEntropyLoss()
        
        test_loss += loss(output, target).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= 250*30
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, 250*30,
        100. * correct / (250*30)))


for epoch in range(1, epochs + 1):
    train_loader=data_loader(batch_size,class_lst,np.arange(750))
    train(epoch)
    test_loader=data_loader(batch_size,class_lst,np.arange(751,1000))
    test()


0


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524590031827/work/aten/src/THC/generic/THCStorage.cu:58

In [14]:
args.cuda=False
args.lr=0.001
args.momentum=0.5


model = LeNet()
if args.cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data=torch.from_numpy(data)
        target=torch.from_numpy(target)
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data=torch.from_numpy(data)
        target=torch.from_numpy(target)
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()


NameError: name 'args' is not defined